### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Rheumatoid_Arthritis/GSE217351'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Validate if we have gene expression data from the background information
is_gene_available = 'gene expression' in background_info.get('!Series_summary', '').lower()

# Determine availability and keys for the required variables based on the content of the sample characteristics
if any('RA' in item for item in sample_characteristics.get(1, [])):
    trait_row = 1

if len(set(sample_characteristics.get(4, []))) > 1:
    age_row = 4

if len(set(sample_characteristics.get(3, []))) > 1:
    gender_row = 3

# Define conversion functions for trait, age, and gender
def convert_trait(value):
    if ':' in value:
        data_value = value.split(':')[1].strip()
        return 1 if data_value == 'RA' else 0 if data_value in ['Healthy', 'iMCD', 'Lymphoma', 'HIV/HHV-8+'] else None
    return None

def convert_age(value):
    try:
        if ':' in value:
            return int(value.split(':')[1].strip())
    except ValueError:
        return None
    return None

def convert_gender(value):
    if ':' in value:
        data_value = value.split(':')[1].strip()
        return 1 if data_value == 'M' else 0 if data_value == 'F' else None
    return None

# Save cohort information with the correct identifiers
save_cohort_info('GSE217351', './preprocessed/Rheumatoid_Arthritis/cohort_info.json', is_gene_available, trait_row is not None)

# Extract clinical features if available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Rheumatoid_Arthritis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Rheumatoid_Arthritis/trait_data/GSE217351.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
